# **Task 1**

In [ ]:
# Colab: first cell
!pip install --upgrade openai jsonschema


In [ ]:
import os
from getpass import getpass

# Prompt (no plain text in notebook)
GROQ_API_KEY = getpass("Enter your GROQ API key (will not be shown): ").strip()
os.environ['GROQ_API_KEY'] = GROQ_API_KEY


Enter your GROQ API key (will not be shown): ··········


In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"   # required for Groq
)
# Note: choose a Groq model that supports tool/function use (see Groq docs).


In [ ]:
from typing import List, Dict
import json

# Each message: {"role": "system"|"user"|"assistant", "content": "..."}

def add_message(history: List[Dict], role: str, content: str):
    history.append({"role": role, "content": content})
    return history

def truncate_by_turns(history: List[Dict], n: int, keep_system=True):
    # Keep system message (if any) and last n messages (user+assistant)
    system = [m for m in history if m["role"] == "system"]
    others = [m for m in history if m["role"] != "system"]
    truncated = others[-n:] if n < len(others) else others
    return (system + truncated) if keep_system else truncated

def truncate_by_length(history: List[Dict], max_chars: int, keep_system=True):
    # Keep the most recent messages until total length <= max_chars
    system = [m for m in history if m["role"] == "system"]
    others = [m for m in history if m["role"] != "system"]
    total = 0
    kept = []
    for m in reversed(others):
        total += len(m["content"])
        if total > max_chars:
            break
        kept.append(m)
    kept = list(reversed(kept))
    return (system + kept) if keep_system else kept


In [ ]:
def maybe_periodic_summarize(history, client, run_counter, k=3, keep_recent_n=2):
    """
    After every k runs, create a summary and replace older messages with a single summary message.
    keep_recent_n: keep last n original user/assistant messages AFTER the summary (so recent context is preserved)
    """
    if run_counter % k != 0:
        return history, None

    summary_text = summarize_history_with_model(client, history)
    # construct new history: keep system (if present), add summary-as-system, then keep last n messages
    system_msgs = [m for m in history if m["role"] == "system"]
    others = [m for m in history if m["role"] != "system"]
    recent = others[-keep_recent_n:] if keep_recent_n > 0 else []
    new_history = system_msgs + [{"role":"system","content":f"[SUMMARY]\n{summary_text}"}] + recent
    return new_history, summary_text


In [ ]:
def summarize_history_with_model(client, history, model="llama-3.3-70b-versatile", max_tokens=400):
    # join history into a single block for summarization
    convo = "\n".join([f"{m['role'].upper()}: {m['content']}" for m in history])
    system = "You are a helpful assistant that produces SHORT, structured summaries (3-5 lines). Focus on facts and decisions."
    user_prompt = f"Summarize the conversation below into a short paragraph with key points and action items:\n\n{convo}"

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role":"system","content":system},
            {"role":"user","content":user_prompt}
        ],
        temperature=0.2,
        max_tokens=max_tokens
    )
    summary = resp.choices[0].message.content   # typical response structure
    return summary.strip()


# Task 2

In [ ]:
contact_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer", "minimum": 0, "maximum": 130}
    },
    "required": ["name", "email"],   # minimal requirement; adjust as assignment expects
    "additionalProperties": False
}


In [ ]:
# Demo samples
samples = [
    {"role":"user","content":"Hi, I'm Alice. My email is alice@example.com. I'm 29 and live in Pune."},
    {"role":"assistant","content":"Thanks Alice — I can help. Do you want to sign up?"},
    {"role":"user","content":"Yes, and also please note my phone: +91-98765-43210."},
    # ... more turns to simulate a long conversation
]

history = []
run_counter = 0
K = 3  # summarize after each 3 runs

for msg in samples:
    add_message(history, msg["role"], msg["content"])
    run_counter += 1
    history, summary = maybe_periodic_summarize(history, client, run_counter, k=K, keep_recent_n=1)
    print(f"--- After run {run_counter} ---")
    if summary:
        print("SUMMARY (stored):", summary)
    print("History length:", len(history))
    # Show truncation examples
    print("Truncate by turns (last 2):", truncate_by_turns(history, 2))
    print("Truncate by length (max 200 chars):", truncate_by_length(history, 200))
    print("\n")


--- After run 1 ---
History length: 1
Truncate by turns (last 2): [{'role': 'user', 'content': "Hi, I'm Alice. My email is alice@example.com. I'm 29 and live in Pune."}]
Truncate by length (max 200 chars): [{'role': 'user', 'content': "Hi, I'm Alice. My email is alice@example.com. I'm 29 and live in Pune."}]


--- After run 2 ---
History length: 2
Truncate by turns (last 2): [{'role': 'user', 'content': "Hi, I'm Alice. My email is alice@example.com. I'm 29 and live in Pune."}, {'role': 'assistant', 'content': 'Thanks Alice — I can help. Do you want to sign up?'}]
Truncate by length (max 200 chars): [{'role': 'user', 'content': "Hi, I'm Alice. My email is alice@example.com. I'm 29 and live in Pune."}, {'role': 'assistant', 'content': 'Thanks Alice — I can help. Do you want to sign up?'}]


--- After run 3 ---
SUMMARY (stored): Key points: Alice introduced herself, sharing her email and age. 
Action items: Sign up Alice with email alice@example.com and phone +91-98765-43210. 
Next steps:

In [ ]:
functions = [
    {
        "name": "extract_contact_info",
        "description": "Extract contact info: name, email, phone, location, age from the chat text",
        "parameters": contact_schema
    }
]

def run_extraction(client, text, model="llama-3.3-70b-versatile"):
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role":"system","content":"You are a JSON extractor. Respond only with the JSON matching the schema when possible."},
            {"role":"user","content": f"Extract contact details from the following message:\n\n{text}"}
        ],
        functions=functions,
        function_call={"name":"extract_contact_info"}  # force function-like output
    )
    # Inspect the response to find the function call args
    choice = resp.choices[0].message
    # Different SDKs represent this slightly differently; try both:
    function_call = None
    if hasattr(choice, "function_call"):
        function_call = choice.function_call
    else:
        function_call = choice.get("function_call") if isinstance(choice, dict) else None

    if function_call:
        args_text = function_call["arguments"] if isinstance(function_call, dict) else function_call.arguments
        try:
            parsed = json.loads(args_text)
        except Exception as e:
            parsed = None
        return parsed, resp
    else:
        # fallback: parse content as JSON
        try:
            parsed = json.loads(choice.content)
        except Exception:
            parsed = None
        return parsed, resp


In [ ]:
from jsonschema import validate, ValidationError

def validate_extracted(parsed):
    try:
        validate(instance=parsed, schema=contact_schema)
        return True, None
    except ValidationError as e:
        return False, str(e)


In [ ]:
sample_chats = [
    "Hello, I'm Rajesh Kumar. My email is rajesh.k@example.com. I'm 34 and live in Bangalore. Phone: +91 90000 11111.",
    "Name: Priya Singh; Email: priya_s@hotmail.com; Age: 27; City: Mumbai; Contact: 9876543210",
    "Hey! It's Sam. Reach me at sam@example.org. 22 y/o. Based in New Delhi."
]

for text in sample_chats:
    parsed, resp = run_extraction(client, text)
    ok, err = validate_extracted(parsed) if parsed else (False, "parse failed")
    print("Input:", text)
    print("Parsed:", parsed)
    print("Validation OK:", ok)
    if not ok:
        print("Validation Error:", err)
    print("---")


Input: Hello, I'm Rajesh Kumar. My email is rajesh.k@example.com. I'm 34 and live in Bangalore. Phone: +91 90000 11111.
Parsed: {'age': 34, 'email': 'rajesh.k@example.com', 'location': 'Bangalore', 'name': 'Rajesh Kumar', 'phone': '+91 90000 11111'}
Validation OK: True
---
Input: Name: Priya Singh; Email: priya_s@hotmail.com; Age: 27; City: Mumbai; Contact: 9876543210
Parsed: {'age': 27, 'email': 'priya_s@hotmail.com', 'location': 'Mumbai', 'name': 'Priya Singh', 'phone': '9876543210'}
Validation OK: True
---
Input: Hey! It's Sam. Reach me at sam@example.org. 22 y/o. Based in New Delhi.
Parsed: {'age': 22, 'email': 'sam@example.org', 'location': 'New Delhi', 'name': 'Sam', 'phone': ''}
Validation OK: True
---
